In [ ]:
import pandas as pd

In [ ]:
!pip install xgboost lightgbm catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
from lightgbm import LGBMRanker
from xgboost import XGBRanker
from catboost import CatBoostRanker

In [ ]:
import os
# Check if the Kaggle API credentials are available
if not os.path.exists('/root/.kaggle/kaggle.json'):
    # If not, create the directory and download the kaggle.json file
    !mkdir -p /root/.kaggle
    !cp kaggle.json /root/.kaggle/
    !chmod 600 /root/.kaggle/kaggle.json
# Now, you can use the Kaggle API to download datasets
!kaggle datasets download -d rabieelkharoua/students-performance-dataset
!unzip students-performance-dataset.zip -d students-performance-dataset # Replace <destination_folder> with your desired path

Dataset URL: https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/66.2k [00:00<?, ?B/s]
100% 66.2k/66.2k [00:00<00:00, 71.2MB/s]
Archive:  students-performance-dataset.zip
  inflating: students-performance-dataset/Student_performance_data _.csv  


In [ ]:
student_performance_df_orig = pd.read_csv('students-performance-dataset/Student_performance_data _.csv')


In [ ]:
student_performance_df_orig.head()

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


In [ ]:
student_performance_df_orig.describe()

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
count,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000,2392.000000
mean,2196.500000,16.468645,0.510870,0.877508,1.746237,9.771992,14.541388,0.301421,2.122074,0.383361,0.303512,0.196906,0.157191,1.906186,2.983696
std,690.655244,1.123798,0.499986,1.028476,1.000411,5.652774,8.467417,0.458971,1.122813,0.486307,0.459870,0.397744,0.364057,0.915156,1.233908
min,1001.000000,15.000000,0.000000,0.000000,0.000000,0.001057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1598.750000,15.000000,0.000000,0.000000,1.000000,5.043079,7.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.174803,2.000000
50%,2196.500000,16.000000,1.000000,0.000000,2.000000,9.705363,15.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.893393,4.000000
75%,2794.250000,17.000000,1.000000,2.000000,2.000000,14.408410,22.000000,1.000000,3.000000,1.000000,1.000000,0.000000,0.000000,2.622216,4.000000
max,3392.000000,18.000000,1.000000,3.000000,4.000000,19.978094,29.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,4.000000,4.000000


In [ ]:
student_performance_df = student_performance_df_orig.drop(['StudentID', 'GradeClass'], axis=1)
student_performance_df

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA
0,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196
1,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915
2,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602
3,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218
4,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,18,1,0,3,10.680555,2,0,4,1,0,0,0,3.455509
2388,17,0,0,1,7.583217,4,1,4,0,1,0,0,3.279150
2389,16,1,0,2,6.805500,20,0,2,0,0,0,1,1.142333
2390,16,1,1,0,12.416653,17,0,2,0,1,1,0,1.803297


In [ ]:

df = student_performance_df_orig.copy()

# Create a simple 'group_id' column, for example we could group by a certain feature like 'Age' or manually
# We'll assume here that students with similar Age are in the same group (this is just for demonstration)
df['group_id'] = df['Age']

# Create ranks based on the GPA within each group (higher GPA should get a higher rank)
df['rank'] = df.groupby('group_id')['GPA'].rank(ascending=False, method='min')

min_rank = df['rank'].min()
max_rank = df['rank'].max()

scaled_rank = 30 * (df['rank'] - min_rank) / (max_rank - min_rank)
df['rank'] = scaled_rank
df['rank'] = df['rank'].astype(int)

df = df.sort_values(by='group_id')

# Define features (X) and target (y)
X = df.drop(columns=['GPA', 'rank', 'group_id'])  # Drop the target column 'GPA', 'rank' and 'group_id'
y = df['rank']

# Create a group array that defines the number of samples in each group
group = df.groupby('group_id').size().values  # This will give the group sizes

# Ensure the sum of group sizes matches the number of rows
assert np.sum(group) == len(df), f"Sum of group sizes {np.sum(group)} does not match data size {len(df)}"

In [ ]:

import lightgbm as lgb

# Create a LightGBM Dataset
train_data = lgb.Dataset(X, label=y, group=group)

# Set parameters for ranking task
params = {
    'objective': 'lambdarank',  # LambdaRank for ranking
    'metric': 'ndcg',           # Use NDCG for evaluation
    'boosting_type': 'gbdt',    # Use Gradient Boosting Decision Trees
    'num_leaves': 31,           # Hyperparameter for leaf nodes in trees
    'learning_rate': 0.05,      # Learning rate
    'verbose': -1               # Suppress LightGBM output
}

# Train the model
num_round = 100
lightgbm_ranker = lgb.train(params, train_data, num_round)

y_pred = lightgbm_ranker.predict(X)

ranked_indices = np.argsort(y_pred)[::-1]  # Sort in descending order to get ranks

predicted_ranks = np.zeros_like(y_pred, dtype=int)
predicted_ranks[ranked_indices] = np.arange(1, len(y_pred) + 1)

print("Predicted Ranks:", predicted_ranks)
print("Predicted Ranks Length:", len(predicted_ranks))


Predicted Ranks: [1735 1484 1526 ...  282 2105 2100]
Predicted Ranks Length: 2392


In [ ]:

import xgboost as xgb

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X, label=y)
dtrain.set_group(group)  # Set the group size

params = {
    'objective': 'rank:pairwise',
    'metric': 'ndcg',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'verbose': 0,
}

num_round = 100
xgboost_ranker = xgb.train(params, dtrain, num_round)

# Predict the ranks for the input data
y_pred = xgboost_ranker.predict(dtrain)

ranked_indices = np.argsort(y_pred)[::-1]  # Sort in descending order to get ranks

predicted_ranks = np.zeros_like(y_pred, dtype=int)
predicted_ranks[ranked_indices] = np.arange(1, len(y_pred) + 1)

print("Predicted Ranks:", predicted_ranks)
print("Predicted Ranks Length:", len(predicted_ranks))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [02:04:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "boosting_type", "metric", "num_leaves", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Predicted Ranks: [1927 1338 1788 ...  311 1941 2392]
Predicted Ranks Length: 2392


In [ ]:

import catboost as cb

# Create CatBoost Pool
train_data = cb.Pool(X, label=y, group_id=df['group_id'])

# Set parameters for ranking task
params = {
    'objective': 'YetiRank',    # Rank objective (for ranking tasks)
    'eval_metric': 'NDCG',       # Use NDCG metric for evaluation
    'learning_rate': 0.05,  # Learning rate
    'iterations': 100,      # Number of boosting iterations
    'depth': 6,             # Depth of trees
    'verbose': 0            # Suppress CatBoost output
}

# Train the model
model = cb.train(train_data, params)

# Predict the ranks for the input data
y_pred = model.predict(X)

# Optionally convert the predicted scores to ranks (if needed)
ranked_indices = np.argsort(y_pred)[::-1]  # Sort in descending order to get ranks

# Now, we create the predicted ranks
predicted_ranks = np.zeros_like(y_pred, dtype=int)
predicted_ranks[ranked_indices] = np.arange(1, len(y_pred) + 1)

print("Predicted Ranks:", predicted_ranks)
print("Predicted Ranks Length:", len(predicted_ranks))

Predicted Ranks: [1292  965 1928 ...  346 2125 2241]
Predicted Ranks Length: 2392
